In [4]:
import json
from pathlib import Path

CURATED_PATH = Path("/Users/james/Documents/obi/Data/Simulations/BBP-curated/assemblies")
RAW_CONFIGS_PATH = Path("/Users/james/Documents/obi/Data/Simulations/BBP-raw/assemblies/")
RAW_INPUT_OUTPUT_PATH = RAW_CONFIGS_PATH / "d21efd45-7740-4268-b06f-e1de35f2b6cf"

sim_length = 127500
ou_start_time = 250
ca_conc = 1.05

In [5]:
def add_relative_ou_stimulus(config, name, node_set, mean, sd, delay, duration):

    config['inputs'][name] = {
            "input_type": "conductance",
            "module": "relative_ornstein_uhlenbeck",
            "delay": delay,
            "duration": duration,
            "reversal": 0,
            "tau": 2.7,
            "mean_percent": mean,
            "sd_percent": sd,
            "node_set": node_set
        }
    
def add_spike_replay(config, sim_length):
    config["inputs"]["Stimulus spikeReplay"] = {
            "module": "synapse_replay",
            "input_type": "spikes",
            "delay": 0,
            "duration": sim_length,
            "spike_file": "input.dat",
            "source": "proj_Thalamocortical_VPM_Source",
            "node_set": "hex_O1"
        }
    
def add_reports(config, sim_length):
  config["reports"] = {
      "Report soma": {
        "cells": "hex_O1",
        "sections": "soma",
        "type": "compartment",
        "compartments": "center",
        "variable_name": "v",
        "unit": "mV",
        "dt": 1.0,
        "start_time": 0,
        "end_time": sim_length
      }
    }
  
def add_connection_overrides(config):
    config["connection_overrides"] = [
            {
                "name": "no_vpm_proj",
                "source": "proj_Thalamocortical_VPM_Source",
                "target": "hex_O1",
                "spontMinis": 0.0,
                "weight": 1.0
            },
            {
                "name": "no_pom_proj",
                "source": "proj_Thalamocortical_POM_Source",
                "target": "hex_O1",
                "spontMinis": 0.0,
                "weight": 1.0
            },
            {
                "name": "init",
                "source": "hex_O1",
                "target": "hex_O1",
                "weight": 1.0
            },
            {
                "name": "disconnect",
                "source": "hex_O1",
                "target": "hex_O1",
                "delay": 0.025,
                "weight": 0.0
            },
            {
                "name": "reconnect",
                "source": "hex_O1",
                "target": "hex_O1",
                "delay": 1000,
                "weight": 1.0
            }
        ]
    
def add_conditions(config, ca_conc):

    config["conditions"] = {
        "extracellular_calcium": ca_conc,
        "v_init": -80.0,
        "mechanisms": {
            "ProbAMPANMDA_EMS": {
                "init_depleted": True,
                "minis_single_vesicle": True
            },
            "ProbGABAAB_EMS": {
                "init_depleted": True,
                "minis_single_vesicle": True
            }
        }
    }




In [8]:
indices_and_seeds = [(0, 1), (1, 19), (2, 31), (3, 39), (4, 47), (5, 54), (6, 63), (7, 78), (8, 80), (9, 99)]

for index, random_seed in indices_and_seeds[0:5]:

    config = {
        "version": 1,
        "run": {
            "dt": 0.025,
            "random_seed": random_seed,
            "tstop": sim_length
        },
        "spike_location": "AIS",
        "network": "../circuit_config.json",
        "node_set": "hex_O1",
        "node_set_file": "node_sets.json",
        "target_simulator": "CORENEURON",
        "output": {
            "output_dir": "./reporting",
            "spikes_file": "spikes.h5"
        },
        "inputs": {},
        "reports": {}
    }

    add_relative_ou_stimulus(config, "Stimulus gExc_L1", "Layer1", 2.993, 1.197, ou_start_time, sim_length)
    add_relative_ou_stimulus(config, "Stimulus gExc_L23E", "Layer23Excitatory", 18.183, 7.273, ou_start_time, sim_length)
    add_relative_ou_stimulus(config, "Stimulus gExc_L23I", "Layer23Inhibitory", 2.587, 1.035, ou_start_time, sim_length)
    add_relative_ou_stimulus(config, "Stimulus gExc_L4E", "Layer4Excitatory", 8.844, 3.538, ou_start_time, sim_length)
    add_relative_ou_stimulus(config, "Stimulus gExc_L4I", "Layer4Inhibitory", 3.13, 1.252, ou_start_time, sim_length)
    add_relative_ou_stimulus(config, "Stimulus gExc_L5E", "Layer5Excitatory", 16.394, 6.558, ou_start_time, sim_length)
    add_relative_ou_stimulus(config, "Stimulus gExc_L5I", "Layer5Inhibitory", 4.761, 1.904, ou_start_time, sim_length)
    add_relative_ou_stimulus(config, "Stimulus gExc_L6E", "Layer6Excitatory", 2.562, 1.025, ou_start_time, sim_length)
    add_relative_ou_stimulus(config, "Stimulus gExc_L6I", "Layer6Inhibitory", 3.026, 1.21, ou_start_time, sim_length)

    add_spike_replay(config, sim_length)
    add_reports(config, sim_length)
    add_connection_overrides(config)
    add_conditions(config, ca_conc)

    curated_sim_dir = CURATED_PATH / str(index)
    curated_sim_dir.mkdir(exist_ok=True, parents=True)
    config_path = curated_sim_dir / Path(f"simulation_config.json")
    with open(config_path, "w") as f:
        json.dump(config, f, indent=4)